In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import itertools

%matplotlib inline
from finrl import config
from finrl import config_tickers
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

/home/henry/miniconda3/envs/finrl/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [2]:
from data_processor.indicator import add_all_indicators


In [3]:
import data_processor.config

In [4]:
price_df = pd.read_parquet("./data/hs300_price.parquet")
fund_df = pd.read_parquet("./data/hs300_fundament.parquet")


In [5]:
df = add_all_indicators(price_df, fund_df)


In [12]:
test = data_split(df, "2009-01-01", "2022-01-01")
b = set(test.loc[3160].tic)

for i in range(3160):
    a = set(test.loc[i].tic)
    b = a & b

In [15]:
test.index.unique()

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3151, 3152, 3153, 3154, 3155, 3156, 3157, 3158, 3159, 3160],
           dtype='int64', length=3161)

In [ ]:
available_tics = df.set_index('date').loc[data_processor.config.FIRST_DAY_OF_2009].tic.tolist()

In [30]:
tmp_list = []
for tic in available_tics:
    tmp_df = df[df['tic'] == tic]
    tmp_list.append(tmp_df)
    
df = pd.concat(tmp_list, ignore_index=True)
    

In [31]:
train = data_split(df, "2009-01-01", "2020-01-01")
trade = data_split(df, "2020-01-01", "2022-01-01")


In [32]:
indicators = data_processor.config.INDICATORS

In [33]:
n_indicators = len(indicators)
n_indicators


17

In [34]:
stock_dimension = len(df.tic.unique())
stock_dimension


139

In [35]:
state_space = 1 + (2 + n_indicators) * stock_dimension
state_space


2642

In [36]:
env_kwargs = {
    "hmax": 300,
    "initial_amount" : 1000000,
    # "initial_list": [10000000] + [0 for i in range(stock_dimension)],
    # buy and sell cost for each stock
    "num_stock_shares" : [0] * stock_dimension,
    "buy_cost_pct": [0.001] * stock_dimension,
    "sell_cost_pct": [0.001] * stock_dimension,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": indicators,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [37]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


### SAC

In [38]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device


In [39]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=10000)

ValueError: could not convert string to float: 

### DDPG

In [ ]:
DDPG_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001
}

agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=5000)

## Backtest

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

In [ ]:
df_account_value.tail()

In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

In [ ]:
from data_processor.ChinaStockDownloader import single_stock_query

baseline_df = single_stock_query("2020-01-01", "2022-01-01", "sh.000300")

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = single_stock_query("2020-01-01", "2022-01-01", "sh.000016")
baseline_df = baseline_df.apply(pd.to_numeric, errors='ignore')
stats = backtest_stats(baseline_df, value_col_name = 'close')

In [ ]:
test_ret = get_daily_return(df_account_value)
base_ret = get_daily_return(baseline_df, 'close')

In [ ]:
print("==============Compare to SZ50===========")
%matplotlib inline

import pyfolio

In [ ]:
test_ret.index = pd.DatetimeIndex(test_ret.index.date)
base_ret.index = pd.DatetimeIndex(base_ret.index.date)

In [ ]:
import quantstats as qs

qs.reports.html(test_ret, base_ret)

In [ ]:
pyfolio.create_full_tear_sheet(returns=test_ret, benchmark_rets=base_ret)